In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('train.csv')
df_train['Status'] = 'Train'
df_train.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags,Status
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12,Train
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8,Train
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2,Train
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0,Train
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4,Train


In [3]:
df_test = pd.read_csv('test.csv')
df_test['Status'] = 'Test'
df_test.head()

,ID,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags,Price,Status
0,44020629,-,VOLKSWAGEN,Golf,2012,Hatchback,No,Diesel,2.0 Turbo,0 km,4,Manual,Front,02-Mar,Left wheel,Grey,10,NaN,Test
1,45784798,-,HYUNDAI,Sonata,2012,Sedan,Yes,Petrol,2.4,26000 km,4,Tiptronic,Front,04-May,Left wheel,Grey,10,NaN,Test
2,45800257,-,NISSAN,Tiida,2005,Sedan,No,Petrol,1.5,168000 km,4,Automatic,Front,04-May,Right-hand drive,Sky blue,8,NaN,Test
3,45797981,975,VOLVO,XC90,2012,Jeep,Yes,Petrol,3.2,143000 km,6,Automatic,4x4,04-May,Left wheel,Blue,12,NaN,Test
4,45814303,-,OPEL,Astra,1993,Hatchback,No,Petrol,1.6,200000 km,4,Manual,Front,04-May,Left wheel,Black,0,NaN,Test


In [4]:
df_total= pd.concat([df_train,df_test],ignore_index = True)
df_total.shape

(27482, 19)

In [5]:
df_total.drop(columns = ['ID'], axis = 1, inplace = True)

In [6]:
df_total['Levy'] = df_total.apply(lambda x: x['Levy'].replace('-','0') if x['Levy'] == '-' else x['Levy'],axis = 1)
df_total['Levy'] = df_total['Levy'].astype(int)

In [7]:
df_total.loc[df_total['Levy'] == 0, 'Levy'] = np.NaN

In [8]:
df_total['Levy'] = df_total['Levy'].fillna(df_total['Levy'].median())

In [9]:
import category_encoders as ce
le = ce.OrdinalEncoder()

In [10]:
df_total["LE_Manufacturer"] = le.fit_transform(df_total["Manufacturer"])
df_total["LE_Manufacturer"] = df_total["LE_Manufacturer"].astype("category")

In [11]:
fe_maker = df_total["Manufacturer"].value_counts().to_dict()
for i in list(set(df_test['Manufacturer'].unique()) - set(df_train['Manufacturer'].unique())): # test - train
    fe_maker[i] = 0

In [12]:
df_total["FE_Manufacturer"] = df_total["Manufacturer"].map(fe_maker)

In [13]:
df_total.drop(['Manufacturer','Model'],axis = 1, inplace  = True)

In [14]:
df_total['Age'] = 2021 - df_total['Prod. year']
df_total.drop('Prod. year',axis=1,inplace = True)

In [15]:
fuel_type_avg_price = df_total[['Fuel type','Price']].groupby('Fuel type',as_index = False).median().rename(columns = {'Price':'fuel_type_avg_price'})

median_encode = dict(dict(fuel_type_avg_price.sort_values(by='fuel_type_avg_price',ascending = False))['Fuel type'])

swap_dict = {value:key for key, value in median_encode.items()}

df_total['OE_Fuel_type'] = df_total['Fuel type'].map(swap_dict)
df_total.drop('Fuel type',axis=1,inplace = True)

In [16]:
df_total['Engine volume'] = df_total['Engine volume'].str[:3]
df_total['Engine volume'] = df_total['Engine volume'].astype(float)

In [17]:
df_total['Mileage'] = df_total['Mileage'].apply(lambda x: str(x).split(" ")[0]).astype(int)

In [18]:
ohe = ['Category','Leather interior','Cylinders','Gear box type','Drive wheels','Doors','Wheel','Airbags']

for i in ohe:
    df_total[i] = df_total[i].astype('category')

In [19]:
# One hot encoding for categorical variables.
ohe = ['Category','Leather interior','Cylinders','Gear box type','Drive wheels','Doors','Wheel','Airbags']
for i in ohe:
        df_total = pd.get_dummies(df_total, prefix = [i],columns=[i],drop_first = True)

In [20]:
df_total.drop('Color',axis=1,inplace = True)

In [21]:
df_total.shape

(27482, 59)

In [22]:
df_total.head()

,Price,Levy,Engine volume,Mileage,Status,LE_Manufacturer,FE_Manufacturer,Age,OE_Fuel_type,Category_Coupe,...,Airbags_7,Airbags_8,Airbags_9,Airbags_10,Airbags_11,Airbags_12,Airbags_13,Airbags_14,Airbags_15,Airbags_16
0,13328.0,1399.0,3.5,186005,Train,1,1433,11,2,0,...,0,0,0,0,0,1,0,0,0,0
1,16621.0,1018.0,3.0,192000,Train,2,1520,10,5,0,...,0,1,0,0,0,0,0,0,0,0
2,8467.0,781.0,1.3,200000,Train,3,1391,15,5,0,...,0,0,0,0,0,0,0,0,0,0
3,3607.0,862.0,2.5,168966,Train,4,1592,10,2,0,...,0,0,0,0,0,0,0,0,0,0
4,11726.0,446.0,1.3,91901,Train,3,1391,7,5,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
from sklearn.preprocessing import StandardScaler

columns_scaled = ['Levy','Engine volume','Mileage','Age','FE_Manufacturer']

df_total[columns_scaled] = StandardScaler().fit_transform(df_total[columns_scaled])
df_total.head()

,Price,Levy,Engine volume,Mileage,Status,LE_Manufacturer,FE_Manufacturer,Age,OE_Fuel_type,Category_Coupe,...,Airbags_7,Airbags_8,Airbags_9,Airbags_10,Airbags_11,Airbags_12,Airbags_13,Airbags_14,Airbags_15,Airbags_16
0,13328.0,1.368180,1.347244,-0.026593,Train,1,-0.708797,0.161990,2,0,...,0,0,0,0,0,1,0,0,0,0
1,16621.0,0.380392,0.779707,-0.026468,Train,2,-0.665728,-0.015543,5,0,...,0,1,0,0,0,0,0,0,0,0
2,8467.0,-0.234060,-1.149918,-0.026302,Train,3,-0.729589,0.872121,5,0,...,0,0,0,0,0,0,0,0,0,0
3,3607.0,-0.024057,0.212171,-0.026946,Train,4,-0.630084,-0.015543,2,0,...,0,0,0,0,0,0,0,0,0,0
4,11726.0,-1.102588,-1.149918,-0.028544,Train,3,-0.729589,-0.548141,5,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
train = df_total[df_total['Status'] == 'Train']
train = train.drop('Status',axis = 1)

test = df_total[df_total['Status'] == 'Test']
test = test.drop('Status',axis = 1)
test = test.drop(['Price'],axis = 1)

In [25]:
train.shape, test.shape

((19237, 58), (8245, 57))

In [26]:
X = train.drop(['Price'],axis = 1)
y = train['Price']

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [28]:
from sklearn.ensemble import ExtraTreesRegressor

extra_reg = ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=3892, verbose=0, warm_start=False)

In [29]:
extra_reg.fit(X_train,y_train)

ExtraTreesRegressor(n_jobs=-1, random_state=3892)

In [30]:
predictions_valid = extra_reg.predict(X_valid)
predictions_valid

array([26343.  , 20089.56, 15524.05, ...,   314.  , 14069.  , 15681.  ])

In [31]:
from sklearn.metrics import mean_squared_log_error,mean_squared_error

def rmsle_sklearn(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))
print('RMSLE',rmsle_sklearn(y_valid,predictions_valid))

RMSLE 0.9949888855000444


In [32]:
predictions = extra_reg.predict(test)
predictions

array([16282.52      , 13674.24      ,  7800.45      , ...,
         552.60869565, 37980.79      ,  3226.93      ])

In [33]:
# generating the submission file in csv format

output = pd.DataFrame({'Price': predictions})
output.to_csv('final_submission.csv',index = False)